In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from skforecast.exceptions import MissingValuesWarning
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.preprocessing import RollingFeatures
from skforecast.ForecasterAutoreg import ForecasterAutoreg

In [34]:
y_datetime = pd.Series(
    [25.3, 29.1, 27.5, 24.3, 2.1, 46.5, 31.3, 87.1, 133.5, 4.3],
    index=pd.date_range('2000-01-01', periods=10, freq='D'),
    name='y', dtype=float
)
exog = pd.DataFrame({
               'col_1': [7.5, 24.4, 60.3, 57.3, 50.7, 41.4, 87.2, 47.4, 14.6, 73.5],
               'col_2': ['a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'b']},
               index = pd.date_range('2000-01-01', periods=10, freq='D')
           )
rolling = RollingFeatures(
    stats=['ratio_min_max', 'median'], window_sizes=4
)

transformer_y = StandardScaler()
transformer_exog = ColumnTransformer(
                        [('scale', StandardScaler(), ['col_1']),
                         ('onehot', OneHotEncoder(), ['col_2'])],
                        remainder = 'passthrough',
                        verbose_feature_names_out = False
                    )

forecaster = ForecasterAutoreg(
                LinearRegression(), 
                lags             = [1, 5], 
                window_features  = rolling,
                transformer_y    = transformer_y,
                transformer_exog = transformer_exog,
                differentiation  = 2
            )
results = forecaster._create_train_X_y(y=y_datetime, exog=exog)
results

(               lag_1     lag_5  roll_ratio_min_max_4  roll_median_4     col_1  \
 2000-01-08 -1.564362 -0.141737             -7.222222      -0.349094  0.040403   
 2000-01-09  1.863585 -0.041996             -0.847826      -0.057745 -1.325790   
 2000-01-10 -0.246728 -0.498706             -0.106061       0.674565  1.127525   
 
             col_2_a  col_2_b  
 2000-01-08      0.0      1.0  
 2000-01-09      0.0      1.0  
 2000-01-10      0.0      1.0  ,
 2000-01-08    1.863585
 2000-01-09   -0.246728
 2000-01-10   -4.609092
 Freq: D, Name: y, dtype: float64,
 ['col_1', 'col_2'],
 ['roll_ratio_min_max_4', 'roll_median_4'],
 ['col_1', 'col_2_a', 'col_2_b'],
 ['lag_1',
  'lag_5',
  'roll_ratio_min_max_4',
  'roll_median_4',
  'col_1',
  'col_2_a',
  'col_2_b'],
 {'col_1': dtype('float64'), 'col_2': dtype('O')})

In [36]:
results[1].to_numpy()

array([ 1.8635851 , -0.24672817, -4.60909217])